<center><h1>Scraping with Selenium</h1>

In [80]:
# importing required libraries
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver

# Function to scrape course details

In [81]:
def scrape_course_details(url, search_word):
    
    
    # Step - 1
    # Get the chrome webdriver and url data
    # Few seconds pause time to wait until page loads
    # Finding element to enter the search text
    # Enter the search key
    # Submit the search key
    
    driver = webdriver.Chrome(executable_path = 'webdrivers/chromedriver')
    driver.get(url)
    time.sleep(5) 
    
    search = driver.find_element_by_xpath("//input[@name='q']")
    time.sleep(2)
    
    search.send_keys(search_word)  
    search.submit()  
    time.sleep(3)
    
    # Step - 2
    # Get the page source after search result page loads
    # Get the last page number for pagination
    # scrape the content from each page
    # store in a datafraome
    
    page_num = 1    # increment page_num for every iteratin in while loop
    pages = True    # while loop runs untill pages is False
    df1 = pd.DataFrame() # empty dataframe to append dataframe outside for loop
    
    while pages:
        
        # gets the page source and beautify it
        main_page_src = driver.page_source
        main_page_soup = BeautifulSoup(main_page_src)
        
        # list of course data from the present page
        course_data = main_page_soup.find_all('div', class_ = 'course-card--main-content--3xEIw')
        
        
        # store course data in a dataframe
        # looping over each course data to extract course names, topics, rating and course length
        course_dict = {}     # to store course data in dictionary
        course_list = []     # to append above dictionary in list
        
        for i in range(len(course_data)):
            subdata = course_data[i]
            
            # course name
            if subdata.find('div', class_ = 'udlite-focus-visible-target') is None:
                course_dict['course_name'] = np.nan
            else:
                course_dict['course_name'] = subdata.find('div', class_ = 'udlite-focus-visible-target').text
            
            # course topics
            if subdata.find('p') is None:
                course_dict['topics'] = np.nan
            else:
                course_dict['topics'] = subdata.find('p').text
            
            # course rating
            if subdata.find('span', class_ = 'udlite-heading-sm') is None:
                course_dict['rating'] = np.nan
            else:
                course_dict['rating'] = subdata.find('span', class_ = 'udlite-heading-sm').text
            
            # course length
            if subdata.find('span', class_ = 'course-card--row--1OMjg') is None:
                course_dict['course_length'] = np.nan
            else:
                course_dict['course_length'] = subdata.find('span', class_ = 'course-card--row--1OMjg').text
            
            # append the dictionary to list and empty the dictionary
            course_list.append(course_dict)
            course_dict = {}
        
        # create a dataframe for course data
        columns = ['course_name', 'topics', 'rating', 'course_length']
        df2 = pd.DataFrame(course_list, columns = columns)
        
        # concat df1 and df2
        df1 = pd.concat([df1, df2])
        
        
        # get the last page number for pagination in the first iteration
        # no need to get the last page number for all the iteration
        # last page number to know when to stop the loop
        if page_num == 1:
            page_nums = [page for page in main_page_soup.find('div', class_ = 'pagination--container--2wc6Z')]
            last_page_num = page_nums[5].text
        
        
        # pagination
        # click the page number to go clicked page
        page_click = driver.find_element_by_xpath("//div[@class = 'pagination--container--2wc6Z']/a[6]")
        page_click.click()
        time.sleep(3)
        
        
        # increment page_num for every iteration
        # when page_num equals to last_page_num (reached the last page)
        page_num += 1
        
        # if page_num == int(last_page_num) + 1:  # if you want to scrape all the pages, uncomment this
        if page_num == int(2) + 1:      # comment it out if and uncomment above condition to scrape all pages
            pages = False
    
    
    # reset index to have continuous index values
    df1.reset_index(inplace = True, drop = True)
    
    return df1

# Calling scrape_course_details to scrape course data

In [73]:
%%time

udemy_url = 'https://www.udemy.com/'
search_word = 'Data Science'

df = scrape_course_details(udemy_url, search_word)

CPU times: user 489 ms, sys: 20 ms, total: 509 ms
Wall time: 23.1 s


In [74]:
df.head()

,course_name,topics,rating,course_length
0,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",4.5,29 total hours
1,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,4.5,44.5 total hours
2,Data Science A-Z™: Real-Life Data Science Exer...,Learn Data Science step by step through real A...,4.6,21.5 total hours
3,"Machine Learning, Data Science and Deep Learni...",Complete hands-on machine learning tutorial wi...,4.5,14.5 total hours
4,Tableau 2020 A-Z: Hands-On Tableau Training fo...,Learn Tableau 2020 for data science step by st...,4.6,8.5 total hours


In [75]:
# df.to_csv('udemy_course_details.csv', index = False)

In [79]:
df

,course_name,topics,rating,course_length
0,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",4.5,29 total hours
1,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,4.5,44.5 total hours
2,Data Science A-Z™: Real-Life Data Science Exer...,Learn Data Science step by step through real A...,4.6,21.5 total hours
3,"Machine Learning, Data Science and Deep Learni...",Complete hands-on machine learning tutorial wi...,4.5,14.5 total hours
4,Tableau 2020 A-Z: Hands-On Tableau Training fo...,Learn Tableau 2020 for data science step by st...,4.6,8.5 total hours
5,R Programming A-Z™: R For Data Science With Re...,Learn Programming In R And R Studio. Data Anal...,4.6,10.5 total hours
6,Statistics for Data Science and Business Analysis,Statistics you need in the office: Descriptive...,4.5,5 total hours
7,Data Science and Machine Learning Bootcamp with R,Learn how to use the R programming language fo...,4.6,18 total hours
8,Python A-Z™: Python For Data Science With Real...,Programming In Python For Data Analytics And D...,4.6,11 total hours
9,Tableau 20 Advanced Training: Master Tableau i...,Master Tableau 20 in Data Science by solving R...,4.7,10 total hours


<center><h1>END</h1>